In [25]:
import numpy as np

In [26]:
class State(object):
    def __init__(self, properties: dict = {}, default: dict = {}):
        N = sum(properties.values())
        self.data = np.zeros(N)
        self.meta = {}

        start_index = 0
        for prop, size in properties.items():
            default_value = default[prop] if prop in default else 0.0

            setattr(self, prop, self.data[start_index:start_index + size])
            getattr(self, prop)[:] = default_value
            
            self.meta[prop] = {
                'index':    start_index,
                'size':     size,
                'default':  default_value
            }

            start_index += size

        def _copy(): 
            copy_state = State(properties, dict([(prop, self.data[meta['index']:meta['index']+meta['size']]) for prop, meta in self.meta.items()]))
            return copy_state

        self.copy = _copy

    def __repr__(self):
        return '\n'.join([f"{prop}[{meta['size']}]: {tuple(getattr(self, prop)[:])}" for prop, meta in self.meta.items()])
        
    def __len__(self):
        return len(self.data)

    def __dict__(self):
        return {'hello': 123 }

    def __getitem__(self, index):
        if isinstance(index, int):
            return self.data[index]
        elif isinstance(index, str):
            return getattr(self, index)[:]
        else:
            raise Exception("Invalid subscript.")

    def __setitem__(self, index, value):
        if isinstance(index, int):
            self.data[index] = value
        elif isinstance(index, str):
            getattr(self, index)[:] = value
        else:
            raise Exception("Invalid subscript.")

    def reset(self):
        for prop, meta in self.meta.items():
            getattr(self, prop)[:] = meta['default']

    def update(self, properties: dict = {}):
        for prop, value in properties.items():
            getattr(self, prop)[:] = value


In [27]:
s = State({'hello': 5, 'world': 3, 'foo': 1}, { 'world': [7,8,9]})

s[8] = 3
t = s.copy()

s['hello'] = [1,2,3,4,5]

print(s)
print(t)   

{'hello': array([0., 0., 0., 0., 0.]), 'world': array([7., 8., 9.]), 'foo': array([3.])}
hello[5]: (1.0, 2.0, 3.0, 4.0, 5.0)
world[3]: (7.0, 8.0, 9.0)
foo[1]: (3.0,)
hello[5]: (0.0, 0.0, 0.0, 0.0, 0.0)
world[3]: (7.0, 8.0, 9.0)
foo[1]: (3.0,)
